In [28]:
%load_ext autoreload 
%autoreload 2

In [29]:
from isaacgym import gymtorch
from isaacgym import gymapi


from isaacgymenvs import train
import isaacgymenvs
from rl_games.torch_runner import Runner

from omegaconf import OmegaConf
from hydra import compose, initialize
import yaml
import torch
import numpy as np
import time
import os

os.environ['DISPLAY'] = ":5"

In [30]:
from urdfpy import URDF
robot = URDF.load(os.path.abspath('../../IsaacGymEnvs/assets/urdf/kuka_allegro_description/allegro_grasp_urdfpy.urdf'))
robot.show()
# default_pos = np.load("allegro_hand_dof_default_pos.npy")
# joint_config = {r: p*0 for r,p in zip(robot.actuated_joint_names, default_pos)}

# robot.show(joint_config)

KeyboardInterrupt: 

In [ ]:
robot.animate()

In [ ]:
robot.base_link

In [ ]:
d = {k: [-.3, .3] for k in robot.actuated_joint_names if 'allegro_mount_joint_' in k}
d.update({k: [-np.pi, np.pi] for k in robot.actuated_joint_names if 'allegro_mount_joint_rev' in k})

In [4]:
with initialize(config_path="cfg", job_name="test_env"):
    cfg = compose(config_name="config", overrides=["task=AllegroHandGrasp", 
                                                   "task.env.observationType=full_state",
                                                   "task.env.objectType=spray_bottle",
                                                   "sim_device=cpu", 
                                                   "test=true",
                                                   "task.env.useRelativeControl=true",
                                                   "checkpoint=./runs/full_state_spray/nn/full_state_spray.pth",
                                                   "num_envs=2"])

env = isaacgymenvs.make(cfg.seed, cfg.task, cfg.num_envs, cfg.sim_device,
                  cfg.rl_device, cfg.graphics_device_id, cfg.headless,
                  cfg.multi_gpu, cfg.capture_video, cfg.force_render, cfg)

/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config': Defaults list is missing `_self_`. See https://hydra.cc/docs/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/hydra/_internal/defaults_list.py:413: UserWarning: In config: Invalid overriding of hydra/job_logging:
Default list overrides requires 'override' keyword.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/defaults_list_override for more information.

  deprecation_warning(msg)
2022-07-21 09:25:16,550 - INFO - logger - logger initialized
/scr1/.pyenv/versions/miniconda3-latest/envs/rlgpu/lib/python3.7/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


Error: FBX library failed to load - importing FBX data will not succeed. Message: No module named 'fbx'
FBX tools must be installed from https://help.autodesk.com/view/FBX/2020/ENU/?guid=FBX_Developer_Help_scripting_with_python_fbx_installing_python_fbx_html
Obs type: full_state
GPU Pipeline can only be used with GPU simulation. Forcing CPU Pipeline.
Not connected to PVD
Physics Engine: PhysX
Physics Device: cpu
GPU Pipeline: disabled
Num hand dofs:  17
Num dofs:  19


## Get camera transform

Saving camera transform from current viewer

```python
import pickle
transform = env.gym.get_viewer_camera_transform(env.viewer, env.envs[0])
pos = transform.p
print("position", pos.x, pos.y, pos.z)

rot = transform.r
print("rotation", rot.x, rot.y, rot.z, rot.w)
pickle.dump(transform, open('allegro_viewer.pkl', 'wb'))
```

```python
import pickle

transform = pickle.load(open('allegro_viewer.pkl', 'rb'))
cam_pos = transform.p
cam_target = gymapi.Vec3()
env.gym.viewer_camera_look_at(env.viewer, None, cam_pos, cam_target)
```

In [6]:
from isaacgym import torch_utils

def get_action(t):
    actions =  torch_utils.unscale(env.shadow_hand_dof_default_pos.repeat(2, 1),
                                   env.shadow_hand_dof_lower_limits, env.shadow_hand_dof_upper_limits).numpy()
    t += 6
    scale = .5 if t > 250 else .25
    actions[:, 0] = - (np.sin(t-6/250 * np.pi))
    return torch.tensor(actions, device=env.device).float()

In [5]:
import pickle
transform = pickle.load(open('allegro_viewer.pkl', 'rb'))
cam_pos = gymapi.Vec3()
cam_pos.x = 0.4
cam_pos.y = -0.75
cam_pos.z = 1.5
cam_target = gymapi.Vec3()
cam_target.x = 0.4
cam_target.y = 0.4
env.gym.viewer_camera_look_at(env.viewer, None, cam_pos, cam_target)

In [7]:
env.reset_idx([0, 1], [0, 1])  # resets
print(env.shadow_hand_dof_pos[0])
action = get_action(-6)
obs, r, done, info = env.step(action)

print(env.object_pos)
print(env.object_rot)
print(env.shadow_hand_dof_pos[:, 0])

tensor([-0.0410, -0.2629,  0.0542,  0.0377, -0.0363, -0.1162,  0.0213,  0.0213,
         0.0378, -0.1116,  0.0378,  0.0378,  0.0213,  0.7429,  0.0062, -0.2792,
        -0.2792])
tensor([[-0.1177, -0.0744,  0.1184],
        [-0.1177, -0.0744,  0.1184]])
tensor([[-0.0543,  0.0188,  0.0020,  0.9983],
        [-0.0543,  0.0188,  0.0019,  0.9983]])
tensor([-0.0406, -0.0406])


In [ ]:
t = 0
hand_vels, palm_pos = [], []
dof_pos = []
# moves hand up and down
while True:
# while t < 500:
    t += 1
    action = get_action(-6)
    obs, r, done, info = env.step(action)
    env_ids = done.nonzero(as_tuple=False).squeeze(-1)
    goal_env_ids = env.reset_goal_buf.nonzero(as_tuple=False).squeeze(-1)
    env.object_dof_vel[:] = 0
    env.object_linvel[:] = 0
    env.object_angvel[:] = 0
    # time.sleep(0.01)
    if done.any():
        env.reset_idx(env_ids, goal_env_ids)
    # palm_1_pos = env.rigid_body_states[0, 1][:3].detach().cpu().numpy().copy()
    # palm_2_pos = env.rigid_body_states[1, 1][:3].detach().cpu().numpy().copy()
    # dof_pos.append(env.shadow_hand_dof_pos[:, 0].detach().cpu().numpy().copy())
    # palm_pos.append((palm_1_pos, palm_2_pos))
    # hand_vels.append(env.shadow_hand_dof_vel[:, 0].detach().cpu().numpy().copy())

KeyboardInterrupt: 

In [26]:
env.rb_forces.shape, env.rigid_body_states.shape

(torch.Size([2, 32, 3]), torch.Size([2, 32, 13]))

In [31]:
env.hand_start_states

tensor([[ 0.0000,  0.0000,  0.2500, -0.3011, -0.6398,  0.2395,  0.6653,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.2500, -0.3011, -0.6398,  0.2395,  0.6653,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [27]:
env.object_rb_handles

tensor([18, 19, 20, 21, 22, 23, 24])

In [12]:
actions = torch.
actions[:, 2] = action[:, 0]

In [15]:
actions = actions[:, :3] * 1/60 * 0.1 * 100000
# env.hand_indices + 1

In [17]:
actions

tensor([[ 0.0000,  0.0000, 12.5545],
        [ 0.0000,  0.0000, 12.5545]])

In [ ]:
env.gym.apply_rigid_body_force_tensors(
    env.sim,
    gymtorch.unwrap_tensor(actions[:, :3]),
    gymtorch.unwrap_tensor(actions[:, 3:6]),

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot([x[0][-1] for x in palm_pos])
plt.plot([x[1][-1] for x in palm_pos])
plt.plot([x[0] for x in dof_pos])
plt.plot([x[1] for x in dof_pos])

In [ ]:
plt.plot(np.array(hand_vels)[:100])